In [239]:
# # Installing necessary dependencies
import sys
import os
# !conda install --yes --prefix {sys.prefix} PIL
# # !{sys.executable} -m pip install ____

# !pip install openpyxl
# !{sys.executable} -m pip install pytesseract
# !{sys.executable} -m pip install pillow
# !{sys.executable} -m pip install python-poppler
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install --upgrade pip
# !conda install -c conda-forge poppler

# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install nltk
# !{sys.executable} -m pip install pandas


In [80]:
# Checking which environment we are working in
# !conda info --envs # check which env we are in
# !conda activate HCP-env # Activate the right environment

In [4]:
# Below, I try to do some text processing and tokenization to see word frequencies

import codecs
import re
import copy
import collections

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division
# import matplotlib
import json


In [5]:
nltk.download('stopwords') # downloads all stopwords to avoid

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
from nltk.corpus import stopwords 

In [7]:
nltk.download('punkt') # downloads the tokenizer we need

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
# Function to tokenize a text file into sentences or phrases
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def get_sentence_counter(path=None, text=None):
    if path:
        path = os.getcwd()
        for file in os.listdir():
            if file.endswith('.txt'):
                with codecs.open(file, 'r') as f: 
                    text = f.read()
    else:
        text = text
    raw_sentences = tokenizer.tokenize(text)
    sentences = []
    for raw_sentence in raw_sentences:
        tmp = raw_sentence.split('\n')
        for x in tmp:
            sentences.append(x)

    return sentences, len(sentences)

In [ ]:
################################################################################
################################################################################
################################################################################
# INDEED

In [17]:
# Working with 'Master_Indeed_Dataset'
path = r'C:\Users\15713\Desktop\Datasets\csv_xlsx\Master_Indeed_Dataset.xlsx'
df = pd.read_excel(path)

In [18]:
df.head(6)

,Title,Company,Location,Links,Salary,Description
0,JUNIOR PENETRATION TESTER,Elevate,Remote,https://indeed.com/viewjob?jk=0a12694d75919f0c,$35 - $50 an hour,JUNIOR PENETRATION TESTER\nwww.elevateconsult....
1,Junior Penetration Tester USA Remote,BreachLock,Floridaâ€¢Remote,https://indeed.com/viewjob?jk=1c98609eab9789cc,None,The Penetration Testing professional should ha...
2,Red Team - Penetration Tester,"Simplicity, LLC.",Remote,https://indeed.com/viewjob?jk=1256f1d0f5d5af0c,"$75,000 - $90,000 a year",Who are we?\nBreachLock is a security startup ...
3,Penetration Tester,Creative Visions,"Washington, DCâ€¢Remote",https://indeed.com/viewjob?jk=c52fc17b016d03d5,None,"Fort Mill, SC, Carrollton TX â€“ Primarily Rem..."
4,Penetration Tester,TechTrueUP,Remote,https://indeed.com/viewjob?jk=d344378f0a869289,$45 - $50 an hour,Summary:\nThe Penetration Tester role will wor...
5,Penetration Tester (remote),Fast AF,"San Francisco, CAâ€¢Remote",https://indeed.com/viewjob?jk=e52b91673b7586f7,None,Job Details:\nIn this vital role you will ensu...


In [19]:
# Creating a dataframe with title as index and description as an attribute
title, description = df['Title'].to_list(), df['Description'].to_list()
indeed_df = pd.DataFrame(data={"Title": title,
                               "Description": description} 
                         )
# removing rows with 'None' for their Job-Title
indeed_df = indeed_df[indeed_df.Title != 'None']

In [20]:
indeed_df.head(3)

,Title,Description
0,JUNIOR PENETRATION TESTER,JUNIOR PENETRATION TESTER\nwww.elevateconsult....
1,Junior Penetration Tester USA Remote,The Penetration Testing professional should ha...
2,Red Team - Penetration Tester,Who are we?\nBreachLock is a security startup ...


In [21]:
# Converting the above dataframe into dictionary for iteration {title: description}
indeed = {}
for (t, d) in zip(title, description): # loops through the list title and description
    if t in indeed and t!='None': # if the job title already exists, we lump the descriptions together
        indeed[t]+=d
    else:
        indeed[t] = d # sets the title as key and the description as the value


In [22]:
# iterating through the Job's to tokenize into sentences/phrases
indeed_tokenized_dict = {}
for key in indeed:
    if key != 'None':
        text = indeed[key]
        # Tokenize into sentence or phrase
        sentence_count, s1 = get_sentence_counter(text=text)
        indeed_tokenized_dict[key] = sentence_count


In [215]:
# Putting in dataframe with keys/job titles as titles and the values as the tokenized sentences just for visual purposes
df_concat = pd.DataFrame()
for k,v in indeed_tokenized_dict.items():
    if k != 'None':
        df = pd.DataFrame({k: v})
        df_concat = pd.concat([df_concat, df], axis=1)

df_concat.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454
JUNIOR PENETRATION TESTER,JUNIOR PENETRATION TESTER,www.elevateconsult.com,ABOUT ELEVATE,Elevate is a premier consulting firm specializ...,"Our team consists of self-managed, high-calibe...","We value exceptional client service, solving a...","We are looking for a motivated, experienced an...",ABOUT THE POSITION,This is a contract to-hire-position which star...,Elevate is looking for a Junior Penetration Te...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Junior Penetration Tester USA Remote,The Penetration Testing professional should ha...,The consultant would perform various penetrati...,The assessments that would be delivered would ...,For the remote assessments the consultant woul...,The consultant should be comfortable identifyi...,The consultant should have a basic understandi...,Must Haves:,"Background in Enterprise network, application,...","1-3 years experience performing application, m...",3-5 years of experience in vulnerability asses...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Red Team - Penetration Tester,Who are we?,BreachLock is a security startup that offers a...,The BreachLock platform leverages both human-p...,BreachLocksâ€™s modern SaaS-based approach red...,"As a result, CIOâ€™s and CISOâ€™s get a single...",The BreachLock platform facilitates collaborat...,Some of our achievements include:,One of the fastest-growing SaaS companies in C...,Cyber Security Innovator for Analysis and Test...,Top 10 Vulnerability Management Solution for 2...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Penetration Tester,"Fort Mill, SC, Carrollton TX â€“ Primarily Rem...",This is a fast track hire (6mth to hire).,See major requirements to consider below:,Â· Position will require 70% Business / 30% Te...,Â· Demonstrates the ability to manage various ...,Â· Provide technical/management leadership on ...,Â· Minimum experience(actual work experience) ...,Â· Ability define security scope while alignin...,Â·,POSITION SUMMARY:,...,Lead engagements and assist less experienced s...,BASIC QUALIFICATIONS,"Bachelorâ€™s degree in cybersecurity, computer...",At least three years of experience in a securi...,Strong attention to detail and problem solving...,As a seven-time Star Tribune Top Workplace we ...,When working at HelpSystems you will have the ...,Check out our website to learn more about Help...,We encourage you to check out our Glassdoor pa...,"As an EEO/Affirmative Action Employer, all qua..."
Penetration Tester (remote),Job Details:,In this vital role you will ensure that flaws ...,"Define, lead and execute red team/blue team ex...",Management of technologies and processes relat...,Develop or participate in the development of b...,"Advise project teams, application owners, and ...",Perform security design consulting in support ...,The creative professional we seek is a self-dr...,Excellent analytical and troubleshooting skills,"Must be team-oriented, placing priority on the...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Remote) Penetration Tester,Our cybersecurity company is seeking a passion...,You will not get bored in our fast-paced work ...,Successful team members strive for excellence ...,Our office is located in the heart of downtown...,To learn more about how you may be able to fit...,If you think you are our ideal candidate apply...,Essential Functions:,Conducting tests on applications and networks:...,"This includes penetration tests, red team test...",Social engineering tests: Conduct remote and o...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cybersecurity Penetration Tester-Web Applications,"Creative Visions is seeking a motivated, caree...",The Penetration Tester will be responsible for...,The ideal candidate will have a basic understa...,"Testing includes web application assessments, ...",Develop and document security evaluation test ...,Provide technical expertise and guidance in de...,Responsible for aligning industry best securit...,Maint

In [338]:
# shows the description
# indeed['SOC Analyst']

In [337]:
# shows the sentences in the description tokenized
# indeed_tokenized_dict['SOC Analyst']

In [184]:
# Here I will try to create 'bag of words' that could be used to identify if a sentence is refering to 
# ability, skill, knowledge or other
bag_A = [' able ', ' ability', 'Ability ', ' abilities', 'perform']
bag_S = ['skill', 'Skill',]
bag_K = ['know', 'known', 'knowledge', 'Knowledge']
bag_C = ['certification', 'Certification']

# Helper function to check whether the flag words are present in the sentence
def check_word(word_list,sentence):
    for x in word_list:
        return True if x in sentence else False

In [236]:
# Now I will sort out the sentences into Ability, skill, knowledge or Certs and save it as a nested dict
Master_indeed_dict = {}
pattern = '[\s|^|^\s][A-Z(][A-Z]+[\s|)|,]'
I_dict = {}
for k in indeed_tokenized_dict.keys():
    K,S,A,C = [],[],[],[]
    for sentence in indeed_tokenized_dict[k]:
        if check_word(bag_K,sentence):
            K.append(sentence)
        if check_word(bag_S,sentence):
            S.append(sentence)
        if check_word(bag_A,sentence):
            A.append(sentence)
        # Here, I tried to use regex to find patterns matching a valid Certificate by first
        # finding a sentence that contains the flag words
        if check_word(bag_C, sentence):
            print(f'\nsentence: {sentence}') #printing the sentence to be checked for valid certs
            match = re.findall(pattern, sentence) #checking words matching our pattern
            print('found certs...')
            for x in match:
                x = re.sub('\W', '', x) #removes any non-word characters (space,symbols)
                C.append(x) #adding cleaned up string to the certs list
                print(f'-{x}')
  
    Master_indeed_dict[k] = {'K':K,
                             'S':S,
                             'A':A,
                             'C':list(set(C))
                            }
    #for visualization purose only:
#     I_dict[k] = K+S+A+C
    

    


sentence: Working towards or more security certifications: OSCP, LPT, CPT, CEH, CEPT, CMWAPT, CRTOP, Comp TIA PenTest+, GPEN
found certs...
-OSCP
-LPT
-CPT
-CEH
-CEPT
-CMWAPT
-CRTOP
-TIA

sentence: Have certifications like CEH or ISO 27001
found certs...
-CEH
-ISO

sentence: Â· Requires Security Certification(s) (i.e., Certified Information Systems Security Professional (CISSP), or Certified Information Security Manage (CISM) or Certified Ethical Hacker (CEH), Offensive Security Certified Professional (OSCP), eLearnSecurity Web application Penetration Tester (eWPT)or other equivalent recognized security certifications (OSCP & eWPT is major plus)
found certs...
-CISSP
-CISM
-CEH
-OSCP
-OSCP

sentence: Information Technology and Security certifications such as Security+, CEH, GIAC, OSCP, or similar preferred.
found certs...
-CEH
-GIAC
-OSCP

sentence: CEH, OSCP or similar certification
found certs...
-OSCP

sentence: Â· Offensive Security Certified Professional (OSCP) and/or GIAC certif

found certs...
-CISSP
-IT

sentence: CISA, CISM, GSEC, CISSP or other security-related certification preferred;
found certs...
-CISM
-GSEC
-CISSP

sentence: An industry-recognized security certification (E.g., Security+, Associate of ISC2, CCNP, CRISC, CISA, CISM, CEH, CISSP, etc.)
found certs...
-CCNP
-CRISC
-CISA
-CISM
-CEH
-CISSP

sentence: Do you have an industry-recognized security certification (E.g., Security+, Associate of ISC2, CCNP, CRISC, CISA, CISM, CEH, CISSP, etc.)?
found certs...
-CCNP
-CRISC
-CISA
-CISM
-CEH
-CISSP

sentence: Security accreditations: CIoTSP, CISSP, ISSAP, CISM, CEH, SANS certifications, CSSA, etc.
found certs...
-CISSP
-ISSAP
-CISM
-CEH
-SANS
-CSSA

sentence: Networking certifications: CompTIA Network+, CCNA, CCNP, etc.
found certs...
-CCNA
-CCNP

sentence: Bachelorâ€™s degree in business administration, computer science, or other related field is required; advanced degree and/or professional certifications (CISA or CIA) is preferred
found certs...
-CIS

In [237]:
I_df =  pd.DataFrame.from_dict({(i,j): Master_indeed_dict[i][j] 
                           for i in Master_indeed_dict.keys() 
                           for j in Master_indeed_dict[i].keys()},
                       orient='index')

In [238]:
I_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
"(JUNIOR PENETRATION TESTER, K)",The Junior Penetration Tester will support ass...,Demonstrated knowledge of vulnerability and pe...,Ability to utilize technical tools (online and...,Intermediate to advance knowledge of various P...,Intermediate to advance knowledge of networkin...,Intermediate to advance knowledge of programmi...,Do you have intermediate to advanced knowledge...,Strong working knowledge of network security t...,We value the safety of each member of our comm...,None,...,None,None,None,None,None,None,None,None,None,None
"(JUNIOR PENETRATION TESTER, S)","Our team consists of self-managed, high-calibe...",Elevate is looking for a Junior Penetration Te...,We seek individuals who enjoy learning to add ...,Transparent reporting which can include a fore...,The candidate must have excellent verbal and w...,Excellent client service and project managemen...,Strong analytical and communications (written ...,The role requires a blend of cybersecurity exp...,Strong problem-solving skills to effectively r...,Be skilled at explaining technical problems su...,...,None,None,None,None,None,None,None,None,None,None
"(JUNIOR PENETRATION TESTER, A)",Our ideal candidate will be able to think and ...,"Must be able to understand client needs, seek ...",The candidate must have excellent verbal and w...,"Be a strong, confident, and exacting writer an...",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
"(JUNIOR PENETRATION TESTER, C)",OSCP,CEH,CRTOP,TIA,LPT,CPT,CEPT,CMWAPT,None,None,...,None,None,None,None,None,None,None,None,None,None
"(Junior Penetration Tester USA Remote, K)",Ability to acquire new knowledge and skills qu...,Our coworkers and clients will â€œknow us by o...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [174]:
# Master_indeed_dict['JUNIOR PENETRATION TESTER']['K']

In [175]:
# Master_indeed_dict['JUNIOR PENETRATION TESTER']['S']

In [ ]:
# Master_indeed_dict['JUNIOR PENETRATION TESTER']['A']

In [203]:
# Master_indeed_dict.keys()
Master_indeed_dict['Cybersecurity Specialist']['C']

['DAWIA', 'IT', 'EOD', 'VRA', 'VEOA', 'DAU', 'AFMAN', 'VA', 'VOW', 'IAW', 'II']

In [204]:
Master_indeed_dict['JUNIOR PENETRATION TESTER']['C']

['OSCP', 'CEH', 'CRTOP', 'TIA', 'LPT', 'CPT', 'CEPT', 'CMWAPT']

In [324]:
# Exporting dictionary as a json file for use during sentence vectorization
data = Master_indeed_dict
with open('Master_indeed_dict.json', 'w') as f:
    json.dump(data, f)

In [ ]:
################################################################################
################################################################################
################################################################################
# SYLLABI

In [32]:
#working with the syllabi data from NICCS
path = r'C:\Users\15713\Desktop\Datasets\csv_xlsx\syllabi_niccs(final).csv'
NICCS = pd.read_csv(path, encoding='cp1252')

In [116]:
NICCS.tail(5)

,College,City,State,Deg,AreaofStudy,Classroom,Online,CourseTitle,Description,LearningObjectives,FrameworkConnections,ConnectwithSpecialtyAreas,WorkRole,WorkRoleID,WorkRoleDesc,Abilities,Knowledge,Skills,Link
191,Florida A&M University,Tallahasse,FL,BS,Information Technology,1,0,Digital Forensics,Technical issues in acquiring computer evidenc...,Concepts in digital forensics including invest...,Investigate,Digital Forensics,Cyber Security Forensics Analyst,IN-FOR-002,Analyzes digital evidence and investigates com...,A0005: Ability to decrypt digital data collect...,K0001: Knowledge of computer networking concep...,"S0032: Skill in developing, testing, and imple...",https://niccs.cisa.gov/training/search/florida...
192,Florida A&M University,Tallahasse,FL,BS,Information Technology,1,0,Digital Forensics,Technical issues in acquiring computer evidenc...,Concepts in digital forensics including invest...,Investigate,Cyber Investigation,Cyber Crime Investigator,IN-INV-001,"Identifies, collects, examines, and preserves ...",A0174: Ability to Find and navigate the dark w...,K0001: Knowledge of computer networking concep...,S0047: Skill in preserving evidence integrity ...,https://niccs.cisa.gov/training/search/florida...
193,Florida A&M University,Tallahasse,FL,BS,Information Technology,1,0,Network Security and Cryptography,The goal of this course is to cover the theore...,Demonstrate an understanding of the theoretica...,Operate and Maintain; Protect and Defend,Cyber Defense Analysis,Cyber Defense Analyst,PR-CDA-001,Uses data collected from a variety of defense ...,A0010: Ability to analyze malware; A0015: Abil...,K0001: Knowledge of computer networking concep...,S0020: Skill in developing and deploying signa...,https://niccs.cisa.gov/training/search/florida...
194,Florida A&M University,Tallahasse,FL,BS,Information Technology,1,0,Network Security and Cryptography,The goal of this course is to cover the theore...,Demonstrate an understanding of the theoretica...,Operate and Maintain; Protect and Defend,Cyber Defense Infrastructure Support,Cyber Defense Infrastructure Support Specialist,PR-INF-001,"Test, deploys, maintains, and administers the ...",A0123: Ability to apply cybersecurity and priv...,K0001: Knowledge of computer networking concep...,S0007: Skill in applying host/network access c...,https://niccs.cisa.gov/training/search/florida...
195,Florida A&M University,Tallahasse,FL,BS,Information Technology,1,0,Network Security and Cryptography,The goal of this course is to cover the theore...,Demonstrate an understanding of the theoretica...,Operate and Maintain; Protect and Defend,Network Services,Network Operations Specialist,OM-NET-001,"Plans, implement, operates netwrok services/sy...",A0052: Ability to operate network equipment in...,K0001: Knowledge of computer networking concep...,S0004: Skill in analyzing network traffic capa...,https://niccs.cisa.gov/training/search/florida...


In [118]:
# Converting Columns needed to list for easier itteration
CourseTitle = NICCS['CourseTitle'].to_list()
WorkRole = NICCS['WorkRole'].to_list()
WorkRoleID = NICCS['WorkRoleID'].to_list()
Tasks = NICCS['WorkRoleDesc'].to_list()
Knowledge = NICCS['Knowledge'].to_list()
Skill = NICCS['Skills'].to_list()
Ability = NICCS['Abilities'].to_list()

In [224]:
# Saving each KSA under each work role as a nested dictionary
Syllabi_dict = {}
niccs_dict = {}
pattern = '[S|A|K][0-9]+[\S][\s]'
for i,WR in enumerate(WorkRole):
    if WR not in Syllabi_dict.keys(): # Avoiding duplicates
        # splitting to get each KSA & IDs, while also removing the A####,K####, S#### tags
        K = re.sub(pattern, '', Knowledge[i]).split(";") 
        S = re.sub(pattern, '', Skill[i]).split(";")
        A = re.sub(pattern, '', Ability[i]).split(";")
        Syllabi_dict[WR] = {'ID': WorkRoleID[i], 
                             'K': K, 
                             'S': S, 
                             'A': A} #Saving as a nested dict
        # for visualization purposes only:
        niccs_dict[WR] = [WorkRoleID[i]] + K + S + A

In [225]:
niccs_concat = pd.DataFrame()
for k,v in niccs_dict.items():
    if k != 'None':
        df = pd.DataFrame({k: v})
        niccs_concat = pd.concat([niccs_concat, df], axis=1)

niccs_concat.T.tail(30)

,0,1,2,3,4,5,6,7,8,9,...,133,134,135,136,137,138,139,140,141,142
Target Developer,AN-TGT-001,Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g.,...","Knowledge of laws, regulations, policies, and...",Knowledge of cybersecurity and privacy princi...,Knowledge of cyber threats and vulnerabilities,Knowledge of specific operational impacts of ...,Knowledge of human-computer interaction princ...,Knowledge of network traffic analysis methods,"Knowledge of concepts, terminology, and opera...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Security Forensics Analyst,IN-FOR-002,Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g.,...","Knowledge of laws, regulations, policies, and...",Knowledge of cybersecurity and privacy princi...,Knowledge of cyber threats and vulnerabilities,Knowledge of specific operational impacts of ...,Knowledge of encryption algorithms,Knowledge of data backup and recovery,Knowledge of incident response and handling m...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Crime Investigator,IN-INV-001,Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g.,...","Knowledge of laws, regulations, policies, and...",Knowledge of cybersecurity and privacy princi...,Knowledge of cyber threats and vulnerabilities,Knowledge of specific operational impacts of ...,Knowledge of intrusion detection methodologie...,Knowledge of system and application security ...,"Knowledge of Insider Threat investigations, r...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Defense Infrastructure Support Specialist,PR-INF-001,Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g.,...","Knowledge of laws, regulations, policies, and...",Knowledge of cybersecurity and privacy princi...,Knowledge of cyber threats and vulnerabilities,Knowledge of specific operational impacts of ...,Knowledge of data backup and recovery,Knowledge of host/network access control mech...,Knowledge of incident response and handling m...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Defense Incident Responder,PR-CIR-001,Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g.,...","Knowledge of laws, regulations, policies, and...",Knowledge of cybersecurity and privacy princi...,Knowledge of cyber threats and vulnerabilities,Knowledge of specific operational impacts of ...,Knowledge of data backup and recovery,Knowledge of business continuity and disaster...,Knowledge of host/network access control mech...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Program Manager,OV-PMA-001,Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g.,...","Knowledge of laws, regulations, policies, and...",Knowledge of cybersecurity and privacy princi...,Knowledge of cyber threats and vulnerabilities,Knowledge of specific operational impacts of ...,Knowledge of information technology (IT) arch...,Knowledge of Risk Management Framework (RMF) ...,Knowledge of resource management principles a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Threat/Warning Analyst,AN-TWA-001,Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g.,...","Knowledge of laws, regulations, policies, and...",Knowledge of cybersecurity and privacy princi...,Knowledge of cyber threats and vulnerabilities,Knowledge of specific operational impacts of ...,Knowledge of human-computer interaction princ...,Knowledge of network traffic analysis methods,"Knowledge of concepts, terminology, and opera...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vulnerability Assessment Analyst,PR-VAM-001,Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g.,...","Knowledge of laws, regulations, policies, and...",Knowledge of cybersecurity and privacy princi...,Knowledge of cyber threats and vulnerabilities,Kn

In [216]:
syllabi_niccs_df =  pd.DataFrame.from_dict({(i,j): Syllabi_dict[i][j] 
                           for i in Syllabi_dict.keys() 
                           for j in Syllabi_dict[i].keys()},
                       orient='index')

In [218]:
syllabi_niccs_df.head(10)

,0
"(Network Operations Specialist, ID)",OM-NET-001
"(Network Operations Specialist, K)",[Knowledge of computer networking concepts and...
"(Network Operations Specialist, S)",[Skill in analyzing network traffic capacity a...
"(Network Operations Specialist, A)",[Ability to operate network equipment includin...
"(Communications Security (COMSEC) Management, ID)",OV-MGT-002
"(Communications Security (COMSEC) Management, K)",[Knowledge of computer networking concepts and...
"(Communications Security (COMSEC) Management, S)",[Skill in determining how a security system sh...
"(Communications Security (COMSEC) Management, A)",[Ability to recognize the unique aspects of th...
"(Cyber Defense Analyst, ID)",PR-CDA-001
"(Cyber Defense Analyst, K)",[Knowledge of computer networking concepts and...


In [120]:
Syllabi_dict.keys()

dict_keys(['Network Operations Specialist', 'Communications Security (COMSEC) Management', 'Cyber Defense Analyst', 'Exploitation Analyst', 'Systems Security Analyst', 'Target Developer', 'Cyber Security Forensics Analyst', 'Cyber Crime Investigator', 'Cyber Defense Infrastructure Support Specialist', 'Cyber Defense Incident Responder', 'Program Manager', 'Threat/Warning Analyst', 'Vulnerability Assessment Analyst', 'Cyber Operator', 'System Testing and Evaluation Specialist', 'Cyber Intel Planner', 'Enterprise Architect', 'Authorizing Official/Designating Representative', 'Information Systems Security Manager', 'Cyber Workforce Developer and Manager', 'Systems Administrator', 'Systems Requirements Planner', 'Information Systems Security Developer', 'Cyber Ops Planner', 'Database Administrator', 'Law Enforcement/CounterIntelligence Forensics Analyst', 'Cyber Defense Forensics Analyst', 'Software Developer', 'Executive Cyber Leadership', 'All Source Collection Manager', 'Knowledge Manag

In [37]:
# Syllabi_dict['Network Operations Specialist']['K']

In [38]:
# Syllabi_dict['Network Operations Specialist']['S']

In [39]:
# Syllabi_dict['Network Operations Specialist']['A']

In [367]:
# Exporting dictionary as a json file for use during sentence vectorization
data = Syllabi_dict
with open('syllabi_NICCS.json', 'w') as f:
    json.dump(data, f)

In [ ]:
################################################################################
################################################################################
################################################################################
# Certs

In [40]:
#working with the Certificate data from NICCS
path = r'C:\Users\15713\Desktop\Datasets\csv_xlsx\cert_list.xlsx' 
Certs = pd.read_excel(path) 

In [41]:
Certs.head(5)

,Cert Name,Cost,Time (hrs),Avg Salary of Recipient,Number of Holders,Knowledge,Skills,Abilities
0,CISSP (Certified Information Systems Security ...,699,40-50,122138.0,92976.0,K0001: Knowledge of computer networking concep...,S0011: Skill in conducting information searche...,A0002: Ability to match the appropriate knowle...
1,CISA (Certified Information Systems Auditor),595,112,132278.0,150000.0,K0001: Knowledge of computer networking concep...,S0027: Skill in determining how a security sys...,A0162: Ability to recognize the unique aspects...
2,CISM (Certified Information Security Manager),760,40-50,148622.0,50000.0,K0001: Knowledge of computer networking concep...,S0027: Skill in determining how a security sys...,A0162: Ability to recognize the unique aspects...
3,Security+,370,45,72244.0,500000.0,K0001: Knowledge of computer networking concep...,S0020: Skill in developing and deploying signa...,A0010: Ability to analyze malware.\nA0013: Abi...
4,CEH (Certified Ethical Hacker),1199,70-75,104813.0,79617.0,K0001: Knowledge of computer networking concep...,S0005: Skill in applying and incorporating inf...,"A0008: Ability to apply the methods, standards..."


In [42]:
cert_name = Certs['Cert Name'].to_list()
c_K = Certs['Knowledge'].to_list()
c_S = Certs['Skills'].to_list()
c_A = Certs['Abilities '].to_list()

In [226]:
# Saving each KSA under each Cert as a nested dictionary
Cert_dict = {}
c_dict = {}
pattern = '[S|A|K][0-9]+[\S][\s]' 
for i,cert in enumerate(cert_name):
    if cert not in Cert_dict.keys(): # Avoiding duplicates
        K = 'None' if pd.isna(c_K[i]) else re.sub(pattern, '', c_K[i]).split("\n")
        S = 'None' if pd.isna(c_S[i]) else re.sub(pattern, '', c_S[i]).split("\n")
        A = 'None' if pd.isna(c_A[i]) else re.sub(pattern, '', c_A[i]).split("\n")

        Cert_dict[cert] = {'K': K, 
                           'S': S, 
                           'A': A} #Saving as a nested dict
        #for visualization purpose only:
        c_dict[cert] = K+S+A

In [229]:
c_dict = pd.DataFrame()
for k,v in Cert_dict.items():
    if k != 'None':
        df = pd.DataFrame({k: v})
        c_dict = pd.concat([c_dict, df], axis=1)

c_dict.T.head(5)

,A,K,S
CISSP (Certified Information Systems Security Professional),[Ability to match the appropriate knowledge re...,[Knowledge of computer networking concepts and...,"[Skill in conducting information searches., Sk..."
CISA (Certified Information Systems Auditor),[Ability to recognize the unique aspects of th...,[Knowledge of computer networking concepts and...,[Skill in determining how a security system sh...
CISM (Certified Information Security Manager),[Ability to recognize the unique aspects of th...,[Knowledge of computer networking concepts and...,[Skill in determining how a security system sh...
Security+,"[Ability to analyze malware., Ability to commu...",[Knowledge of computer networking concepts and...,[Skill in developing and deploying signatures....
CEH (Certified Ethical Hacker),"[Ability to apply the methods, standards, and ...",[Knowledge of computer networking concepts and...,[Skill in applying and incorporating informati...


In [231]:
cert_niccs_df =  pd.DataFrame.from_dict({(i,j): Cert_dict[i][j] 
                           for i in Cert_dict.keys() 
                           for j in Cert_dict[i].keys()},
                       orient='index')

In [232]:
cert_niccs_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,333,334,335,336,337,338,339,340,341,342
"(CISSP (Certified Information Systems Security Professional), K)",Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g., ...","Knowledge of laws, regulations, policies, and ...",Knowledge of cybersecurity and privacy princip...,Knowledge of cyber threats and vulnerabilities.,Knowledge of specific operational impacts of c...,Knowledge of application vulnerabilities.,Knowledge of cyber defense and vulnerability a...,Knowledge of cryptography and cryptographic ke...,Knowledge of organization's enterprise informa...,...,None,None,None,None,None,None,None,None,None,None
"(CISSP (Certified Information Systems Security Professional), S)",Skill in conducting information searches.,"Skill in conducting knowledge mapping (e.g., m...",Skill in creating policies that reflect system...,Skill in discerning the protection needs (i.e....,Skill in the measuring and reporting of intell...,Skill in using knowledge management technologies.,"Skill in administrative planning activities, t...",Skill in preparing plans and related correspon...,Skill in communicating with all levels of mana...,Skill to anticipate new security threats.,...,None,None,None,None,None,None,None,None,None,None
"(CISSP (Certified Information Systems Security Professional), A)",Ability to match the appropriate knowledge rep...,Ability to determine the validity of technolog...,Ability to assess and forecast manpower requir...,"Ability to develop policy, plans, and strategy...",Ability to leverage best practices and lessons...,Ability to oversee the development and update ...,Ability to apply critical reading/thinking ski...,Ability to coordinate cyber operations with ot...,Ability to exercise judgment when policies are...,Ability to identify external partners with com...,...,None,None,None,None,None,None,None,None,None,None
"(CISA (Certified Information Systems Auditor), K)",Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g., ...","Knowledge of laws, regulations, policies, and ...",Knowledge of cybersecurity and privacy princip...,Knowledge of cyber threats and vulnerabilities.,Knowledge of specific operational impacts of c...,Knowledge of encryption algorithms,Knowledge of business continuity and disaster ...,Knowledge of cybersecurity and privacy princip...,Knowledge of incident response and handling me...,...,None,None,None,None,None,None,None,None,None,None
"(CISA (Certified Information Systems Auditor), S)",Skill in determining how a security system sho...,Skill in using Virtual Private Network (VPN) d...,Skill in using Public-Key Infrastructure (PKI)...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
"(CISA (Certified Information Systems Auditor), A)",Ability to recognize the unique aspects of the...,Ability to interpret Communications Security (...,Ability to identify the roles and responsibili...,Ability to manage Communications Security (COM...,Ability to identify types of Communications Se...,Ability to recognize the importance of auditin...,Ability to Identify the requirements of In-Pro...,Ability to recognize the unique aspects of the...,None,None,...,None,None,None,None,None,None,None,None,None,None
"(CISM (Certified Information Security Manager), K)",Knowledge of computer networking concepts and ...,"Knowledge of risk management processes (e.g., ...","Knowledge of laws, regulations, policies, and ...",Knowledge of cybersecurity and privacy princip...,Knowledge of cyber threats and vulnerabilities.,Knowledge of specific operational impacts of c...,Knowledge of encryption algorithms,Knowledge of business continuity and disaster ...,Knowledge of cybersecurity and privacy princip...,Knowledge of incident response and handling me...,...,None,None,None,None,None,None,None,None,None,None
"(CISM (Certified Information Security Manager), S)",Skill in determining how a security syste

In [44]:
# Cert_dict.keys()

In [45]:
# Cert_dict['CISA (Certified Information Systems Auditor)']['A']

In [238]:
# Exporting dictionary as a json file for use during sentence vectorization
data = Cert_dict
with open('Certs.json', 'w') as f:
    json.dump(data, f)

In [ ]:
############## HERE I JUST WANTED TO TRY TO GROUP SIMILAR JOB-TITLES(JOB POSTINGS) AND JOB-ROLES(SYLLABI) TOGETHER
############## USING "THE MOST COMMON WORD" TECHNIQUE

In [114]:

# function that returns the number of common words in two sentences

def common_words_count(s1, s2):
    count = 0
    # Convert sentence to all lower case and split them into words
    try:
        l1 = s1.lower().split(' ') if s1==s1 else None
        l2 = s2.lower().split(' ') if s2==s2 else None

        #Lemmetization if necessary (but for now, i will try it without)

        if l1 is not None:
            for w1 in l1:
                if l2 is not None and w1 in l2:
                    count+=1
    except:
        pass
    return count



In [214]:
min_count = 3 #Here, it is possible to control the minimum common words shared
# But this will be automatically reassigned if the total number of words are 3 or less,
# and if we try to match more common words than the total number of words that are in the list

similar_jobs = {}
for K in Syllabi_dict.keys():
        tmp = min_count
        similar_jobs[K] = []
        print(f'Similar job-titles to: \"{K}\"')
        K_count = len(K.split(' '))
#         print(f'{K_count}, min_count: {min_count}')
        if  min_count > K_count or K_count < 3: # assumption: A job title can only be uniquely described by atleast 3 words
            # and, We can not have a greater common words count than total words count
            print(f"[Invalid \'min_count: {min_count}\'. It has been adjusted to {K_count}]")
            min_count = K_count
            
        for k in Master_indeed_dict.keys():
            if K is not None and k is not None:
                if common_words_count(K,k) >= min_count:
                    similar_jobs[K].append(k)
                    print(f'\n\t\t - "{k}\" ')
        min_count = tmp
        if len(similar_jobs[K]) == 0:
            print('\n \t\t!No similar job-title was found!!! ')
        print("\n\nMoving to the next Work-Role...")
print("\n\t\t!End of Work-Role!!!")
        
                    
        

Similar job-titles to: "Network Operations Specialist"

		 - "Network Security Operations Specialist - Washington DC, Moun..." 


Moving to the next Work-Role...
Similar job-titles to: "Communications Security (COMSEC) Management"

 		!No similar job-title was found!!! 


Moving to the next Work-Role...
Similar job-titles to: "Cyber Defense Analyst"

		 - "Entry-Level Cyber Defense Analyst" 


Moving to the next Work-Role...
Similar job-titles to: "Exploitation Analyst"
[Invalid 'min_count: 3'. It has been adjusted to 2]

 		!No similar job-title was found!!! 


Moving to the next Work-Role...
Similar job-titles to: "Systems Security Analyst"

		 - "Systems / Security Analyst" 


Moving to the next Work-Role...
Similar job-titles to: "Target Developer"
[Invalid 'min_count: 3'. It has been adjusted to 2]

 		!No similar job-title was found!!! 


Moving to the next Work-Role...
Similar job-titles to: "Cyber Security Forensics Analyst"

		 - "Cyber Security Analyst - Expert" 

		 - "Senio

In [166]:
similar_jobs["Cyber Security Forensics Analyst"]

['Cyber Security Analyst - Expert',
 'Senior Cyber Security Analyst',
 'Cyber Security Analyst',
 'Sr. Cyber Security Analyst',
 'Cyber Security Analyst Associate SOC Tier 1 ITC#3073',
 'Junior Cyber Security Assurance Analyst',
 'Cyber Security Vulnerability Analyst (Remote Okay)',
 'Cyber Security Analyst (Entry-Level)',
 'Cyber Analyst (Junior) (Computer Sys Security Anlyst 1)',
 'Cyber Security Hunt Analyst',
 'Cyber Security Monitoring and Response Analyst',
 'Cyber Security Analyst (Multiple Locations)',
 'AFS - Cyber Security Analyst',
 'IT Cyber Security Analyst I',
 'Jr. Cyber Security Analyst',
 'Junior Cyber Security Analyst',
 'CIP Cyber & Physical Security Analyst I',
 'Analyst I - Cyber Security']

In [ ]:
# Next Steps:
# 1. Lump KSAs of similar jobs together and save as a dictionary, where the keys are either Work-Role/WorkRole-ID  
# 2. Using cosine similarity, compare the KSAs of Work-Role/WorkRole-ID in the new dictionary with the KSAs of the same 
#    Work-Role/WorkRole-ID in syllabi
#
# NEXT, WORKING WITH CERTS AND JOB-POSTINGS....
# 
